<a href="https://colab.research.google.com/github/jmestanza/time_stretching/blob/master/code/vad_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%cd /content
from IPython.display import clear_output
#!git clone https://github.com/jmestanza/time_stretching.git

#imports
#!pip install librosa
!pip install webrtcvad
import webrtcvad
!pip install --upgrade librosa
import librosa 
import scipy
import numpy as np
from IPython.display import Audio

clear_output()

In [0]:
#local modules
!wget https://raw.githubusercontent.com/jmestanza/time_stretching/master/code/detect_pitch.py
!wget https://raw.githubusercontent.com/jmestanza/time_stretching/master/code/modified_psola.py
!wget https://raw.githubusercontent.com/jmestanza/time_stretching/master/code/vad.py

from modified_psola import modified_psola
from detect_pitch import get_fundamental_frequency
from vad import float2pcm, pcm2float, frame_generator, join_vad_frames, get_vad_frames
from vad import Frame

#audios
!wget "https://raw.githubusercontent.com/vicaub/Pitch-Estimation-and-Voicing-Detection/master/data/fda_ue/rl022.wav"
#!wget "https://raw.githubusercontent.com/jmestanza/time_stretching/master/LPC%2C%20PSOLA%20and%20Pitch%20detection%20speed%20up/input/ahh_without_silences_8k_norm.wav"

clear_output()

In [3]:
fs = 48000

#os.chdir("/content/time_stretching/code/input/")

audio_file = "/content/rl022.wav"
test, _ = librosa.load(audio_file, sr=fs, mono=True)
audio_int = float2pcm(test)


# necesito al menos dos periodos de la señal
#200 = (50+25)+50+(50+25)
#L = 50 # muestras de la ventana
#K = L//2
# tamaño total por observacion
# total = 4*L

#quiero tener 20 milisegundos de señal
T = 20*1e-3
samples = int(T * fs) 
L = samples//4
K = L


Audio(data=audio_int, rate=fs)

In [16]:
#voy a unir todo

vad = webrtcvad.Vad(3)    #0 is the least aggressive about filtering out non-speech, 3 is the most aggressive; 3 pone mas frames en no-sonoros

#divido el audio en frames de 20 milisegundos
frames = frame_generator(20, audio_int, fs)
frames = list(frames)

speech, not_speech, is_frame_speech = get_vad_frames(frames, vad, fs)


speed = 2.0


#primero sonoros
lpc_coef = librosa.lpc(speech, 12)
voiced_error = scipy.signal.lfilter(lpc_coef, [1], speech) # obtenemos la señal error a traves del filtrado

f0,sample_f0 = get_fundamental_frequency(voiced_error, K, L, fs)

#print("Frecuencia fundamental : ",f0," ", sample_f0)
voiced_audio = modified_psola(voiced_error, sample_f0, speed)

voiced_filt = scipy.signal.lfilter([1], lpc_coef, voiced_audio)


#final = join_vad_frames(speech, nspeech, is_frame_speech)

Audio(data=voiced_filt,rate=fs)
